To run this notebook is it required to have ollama running in port 11434 (default port).

You can run it on windows, mac or linux, download from https://ollama.com/download 

Or on a docker container https://ollama.com/blog/ollama-is-now-available-as-an-official-docker-image

Two models are used:
- qwen2.5:0.5b
- granite4:1b

Download them using the command:
```
ollama pull qwen2.5:0.5b
ollama pull granite4:1b
```

### Test Ollama connection

In [5]:
# https://docs.langchain.com/oss/python/integrations/chat/ollama

from langchain_ollama import ChatOllama
MODEL="qwen2.5:0.5b"    # ollama pull qwen2.5:0.5b

client = ChatOllama(model=MODEL,
                    temperature=0
                    )

messages = [("system", "Just reply with two words"), 
            ("human", "Hi"),
            ]

response = client.invoke(messages)
print(response.content)

Hello!


### Tutorial LangChain but updated to use Ollama from https://docs.langchain.com/oss/python/langchain/quickstart

In [6]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
import os
load_dotenv()  # Loads .env variables as environment variables

os.environ["LANGCHAIN_TRACING"] = "false"
model = init_chat_model(model=MODEL, model_provider="ollama")
response = model.invoke("Why do parrots talk? in 5 words")
print(response.content)

Parrots can mimic sounds and expressions due to their evolutionary adaptation to live in human societies.


In [12]:
# Basic streaming response
for chunk in model.stream("Why do parrots have colorful feathers? in 5 words"):
    print(chunk.text, end="|", flush=True)

Par|rots|'| vibrant| plum|age| is| due| to| pig|ments| and| ultr|aviolet| light| absorption|.|||

### Build a basic agent Ollama

In [7]:
from langchain.agents import create_agent

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_agent(
    model=init_chat_model(model=MODEL, model_provider="ollama"),
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='9346f88f-8f50-48b6-b33f-dfd0c29e216c'),
  AIMessage(content="To provide you with the current weather forecast for San Francisco (SF), I'll need to know your location. Could you please tell me the name of the city where you are? If you don't have a specific name, just give me the short address or city abbreviation.", additional_kwargs={}, response_metadata={'model': 'qwen2.5:0.5b', 'created_at': '2025-12-14T21:24:32.0375776Z', 'done': True, 'done_reason': 'stop', 'total_duration': 922491600, 'load_duration': 180212300, 'prompt_eval_count': 146, 'prompt_eval_duration': 19141400, 'eval_count': 56, 'eval_duration': 634106400, 'logprobs': None, 'model_name': 'qwen2.5:0.5b', 'model_provider': 'ollama'}, id='lc_run--019b1ec0-330a-7cb2-97ce-b8a89100af20-0', usage_metadata={'input_tokens': 146, 'output_tokens': 56, 'total_tokens': 202})]}

In [8]:
from typing import List

from langchain.messages import AIMessage
from langchain.tools import tool
from langchain_ollama import ChatOllama


@tool
def validate_user(user_id: int, addresses: List[str]) -> bool:
    """Validate user using historical addresses.

    Args:
        user_id (int): the user ID.
        addresses (List[str]): Previous addresses as a list of strings.
    """
    return True


llm = ChatOllama(model=MODEL,
                 validate_model_on_init=True,
                 temperature=0,
                ).bind_tools([validate_user])

result = llm.invoke(
    "Could you validate user 123? They previously lived at "
    "123 Fake St in Boston MA and 234 Pretend Boulevard in "
    "Houston TX."
)

if isinstance(result, AIMessage) and result.tool_calls:
    print(result.tool_calls)

[{'name': 'validate_user', 'args': {'addresses': ['123 Fake St', '234 Pretend Boulevard'], 'user_id': 123}, 'id': '2d5eb458-8152-45a6-a443-176a5047963c', 'type': 'tool_call'}]


In [9]:
# Using a non reasoning model
from langchain.messages import HumanMessage
from langchain_ollama import ChatOllama

llm = ChatOllama(model=MODEL)

messages = [
    HumanMessage("What is 3^3?"),
]

response = llm.invoke(messages)
print(response.content)

The expression "3^3" represents the mathematical operation of exponentiation.

In math, exponentiation means raising a number to another power, which is denoted by an "exponentiation sign" (x) and followed by the base number with a caret (^), which shifts the original base into the new position or power. For example:

- 2^3 = 2 * 2 * 2
- 5^2 = 5 * 5

In this case, 3 is raised to the power of itself, which is just 3:

3^3 = 3 * 3 * 3 = 27

So, "3^3" equals 27.


In [11]:
# Using a reasoning model
from langchain.messages import HumanMessage
from langchain_core.messages import ChatMessage
from langchain_ollama import ChatOllama

llm = ChatOllama(model="granite4:1b")

messages = [
    ChatMessage(role="control", content="thinking"),
    HumanMessage("What is 3^3?"),
]

response = llm.invoke(messages)
print(response.content)

The expression 3^3 refers to the mathematical operation of exponentiation, which means multiplying the base (in this case, 3) by itself for a specified number of times equal to the exponent (also in this case, 3). 

So, 3^3 equals:

3 * 3 * 3 = 27

Therefore, the answer is 27.


### Build a real-world agent

This example is available in https://docs.langchain.com/oss/python/langchain/quickstart using Claude Sonnet 4.5.
For this example I used Ollama an local model 

In [71]:
MODEL='granite4:1b'       # ollama pull granite4:1b

In [72]:
# Define the system prompt
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.
            You have access to two tools:
            - get_weather_for_location: use this to get the weather for a specific location
            - get_user_location: use this to get the user's location
            If a user asks you for the weather, make sure you know the location. If you can tell 
            from the question that they mean wherever they are, use the get_user_location tool 
            to find their location."""

In [ ]:
# Create tools
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime

@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    print(f'get_weather_argument : {city}')
    if city == "Florida":
        return f"It's always sunny in {city}!"
    return f"It's raining in {city}!"

@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    print(f'get_user_location_argument : {user_id}')
    return "Florida" if user_id == "1" else "SF"

In [102]:
# Configure your model
from langchain.chat_models import init_chat_model

model=init_chat_model(model=MODEL, 
                      model_provider="ollama",
                      temperature=0.5,
                      )


In [103]:
# Add memory
from langgraph.checkpoint.memory import InMemorySaver
checkpointer = InMemorySaver()

In [104]:
def responseFormat(response):
    for item in response["messages"]:
        if isinstance(item, HumanMessage):
            if item.content:
                print(f"Query : {item.content}")
        if isinstance(item, AIMessage):
            if item.content:
                print(f"AI response: {item.content}")

In [105]:
# Create and run model
from langchain.messages import AIMessage
from langchain.messages import HumanMessage

agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    checkpointer=checkpointer
)

In [106]:
# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "Tell me my location and what the weather outside is."}]},
    config=config,
    context=Context(user_id="1")
)

# # Note that we can continue the conversation using the same `thread_id`.
response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config=config,
    context=Context(user_id="1")
)

responseFormat(response)

get_user_location_argunment : 1
get_weather_argunment : Florida
Query : Tell me my location and what the weather outside is.
AI response: The weather outside is currently sunny and warm, perfect for a day of sunshine!
Query : thank you!
AI response: You're welcome! If you need anything else, just let me know.


In [107]:
# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "2"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "Tell me my location and what the weather outside is."}]},
    config=config,
    context=Context(user_id="2")
)

# # Note that we can continue the conversation using the same `thread_id`.
response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config=config,
    context=Context(user_id="2")
)

responseFormat(response)

get_user_location_argunment : 2
get_weather_argunment : <user's location>
Query : Tell me my location and what the weather outside is.
AI response: The weather is rainy in SF! Looks like it's time to grab an umbrella.
Query : thank you!
AI response: You're welcome! If you need anything else, just let me know.
